<a href="https://colab.research.google.com/github/DanielMollerEriksen/danieleriksen.github.io/blob/main/Matching_UK_MPs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install fuzzywuzzy

In [2]:
import pandas as pd
from fuzzywuzzy import process


/usr/local/lib/python3.10/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [4]:
maml_df = pd.read_csv('maml_unique_MPs_UK.csv')
realname_df = pd.read_csv('unique_MPs_Twitter_UK.csv')


In [7]:
# Define a fuzzy matching function
def fuzzy_match(row, choices, scorer, threshold=85):
    best_match = process.extractOne(row['full_name'], choices, scorer=scorer)
    if best_match[1] >= threshold:
        return best_match[0]
    return None


In [10]:
from fuzzywuzzy import fuzz


In [12]:
maml_df

,country,actor_id,firstName,lastName,middleNames,partyId,partyNameShort
0,uk,A_UK_1000,Adam,Afriyie,NaN,P_773,Con
1,uk,A_UK_1001,Adam,Holloway,NaN,P_773,Con
2,uk,A_UK_1002,Adam,Ingram,NaN,P_1556,Lab
3,uk,A_UK_1003,Adam,Price,NaN,P_311,Plaid
4,uk,A_UK_1004,Adrian,Bailey,NaN,P_1556,Lab
...,...,...,...,...,...,...,...
1429,uk,A_UK_3001,Órfhlaith,Begley,NaN,P_689,SF
1430,uk,A_UK_3002,Jane,Dodds,NaN,P_659,Lib
1431,uk,A_UK_3003,Lisa,Forbes,NaN,P_1556,Lab
1432,uk,A_UK_3004,Janet,Daby,NaN,P_1556,Lab


In [13]:
maml_df['full_name'] = maml_df['firstName'].str.cat(maml_df[['middleNames', 'lastName']], sep=' ', na_rep='').str.strip()


In [14]:
# Apply fuzzy matching to find best matches
choices = realname_df['real_name'].tolist()
maml_df['matched_name'] = maml_df.apply(fuzzy_match, choices=choices, scorer=fuzz.token_sort_ratio, axis=1)


In [15]:
maml_df

,country,actor_id,firstName,lastName,middleNames,partyId,partyNameShort,full_name,matched_name
0,uk,A_UK_1000,Adam,Afriyie,NaN,P_773,Con,Adam Afriyie,Adam Afriyie
1,uk,A_UK_1001,Adam,Holloway,NaN,P_773,Con,Adam Holloway,None
2,uk,A_UK_1002,Adam,Ingram,NaN,P_1556,Lab,Adam Ingram,None
3,uk,A_UK_1003,Adam,Price,NaN,P_311,Plaid,Adam Price,None
4,uk,A_UK_1004,Adrian,Bailey,NaN,P_1556,Lab,Adrian Bailey,Adrian Bailey
...,...,...,...,...,...,...,...,...,...
1429,uk,A_UK_3001,Órfhlaith,Begley,NaN,P_689,SF,Órfhlaith Begley,Órfhlaith Begley
1430,uk,A_UK_3002,Jane,Dodds,NaN,P_659,Lib,Jane Dodds,None
1431,uk,A_UK_3003,Lisa,Forbes,NaN,P_1556,Lab,Lisa Forbes,None
1432,uk,A_UK_3004,Janet,Daby,NaN,P_1556,Lab,Janet Daby,Janet Daby


In [16]:
# Merge the dataframes based on the matched names
merged_df = pd.merge(maml_df, realname_df, left_on='matched_name', right_on='real_name', how='outer')


In [17]:
merged_df

,country,actor_id,firstName,lastName,middleNames,partyId,partyNameShort,full_name,matched_name,Unnamed: 0,real_name,party
0,uk,A_UK_1000,Adam,Afriyie,NaN,P_773,Con,Adam Afriyie,Adam Afriyie,268.0,Adam Afriyie,Conservatives
1,uk,A_UK_1001,Adam,Holloway,NaN,P_773,Con,Adam Holloway,None,NaN,NaN,NaN
2,uk,A_UK_1002,Adam,Ingram,NaN,P_1556,Lab,Adam Ingram,None,NaN,NaN,NaN
3,uk,A_UK_1003,Adam,Price,NaN,P_311,Plaid,Adam Price,None,NaN,NaN,NaN
4,uk,A_UK_1005,Adrian,Flook,NaN,P_773,Con,Adrian Flook,None,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1442,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,230.0,Stewart Malcolm McDonald,Scottish National Party
1443,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,276.0,Robert Neill,Conservatives
1444,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,357.0,Naz Shah,Labour
1445,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,418.0,David T. C. Davies,Conservatives


In [21]:
# Save the merged dataset
merged_df.to_csv('merged_full_politicians_UK.csv', index=False)


In [23]:
# Save the merged dataset as an Excel file
merged_df.to_excel('new_merged_full_politicians_UK.xlsx', index=False)
